<a href="https://colab.research.google.com/github/wojciechsadlik/MGU-CGANMusicConverter/blob/main/raw_audio_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pickle
import numpy as np
from glob import glob
from keras.utils.np_utils import to_categorical

In [3]:
def get_data(file_list):
  def load_into(filename, x, y):
    with open(filename, 'rb') as f:
      pkl_file = pickle.load(f)
      x.append(pkl_file['audio'])
      y.append(int(pkl_file['class_id']))

  x, y = [], []
  for filename in file_list:
    load_into(filename, x, y)

  return np.array(x), np.array(y)

In [1]:
from keras.layers.pooling import MaxPooling1D
from keras.backend import in_top_k
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras import regularizers, Input
from tensorflow.keras.layers import BatchNormalization
from keras.layers.core import Activation
from keras.layers import LeakyReLU, Add, Dense, Flatten
from tensorflow.keras.utils import plot_model

AUDIO_LENGTH = 110250

def add_Res1D(y, kernel_size, filters, strides=1, downsample=False):
  y_shortcut = y

  y = Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = LeakyReLU()(y)
  y = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(y)
  y = BatchNormalization()(y)

  if downsample or strides > 1:
    y_shortcut = Conv1D(filters=filters, kernel_size=1, strides=strides, padding='same')(y_shortcut)

  y = Add()([y_shortcut, y])
  y = LeakyReLU()(y)

  return y


def build_discriminator(input_shape=[AUDIO_LENGTH, 1]):

  X_input = Input(input_shape)
  
  y = Conv1D(128, kernel_size=3, strides=3, padding='same')(X_input)
  y = add_Res1D(y, 3, 128)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 128)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 256, downsample=True)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 256)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 256)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 256)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 256)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 256)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = add_Res1D(y, 3, 512, downsample=True)
  y = MaxPooling1D(pool_size=3, strides=3)(y)
  y = Conv1D(512, kernel_size=1, strides=1, padding='same')(y)
  y = Flatten()(y)
  y = Dense(1, activation='sigmoid')(y)

  model = Model(inputs = X_input, outputs = y)
  return model

In [5]:
discriminator = build_discriminator()

In [6]:
#discriminator.summary()

In [28]:
from keras.layers.convolutional import Conv1DTranspose
from keras.layers import Reshape

def build_generator(input_shape=(98, )):

  X_input = Input(input_shape)
  
  y = Dense(input_shape[0] * 128)(X_input)
  y = Reshape((input_shape[0], 128))(y)
  y = Conv1DTranspose(128, kernel_size=7, strides=5, padding='same')(y)
  y = LeakyReLU()(y)
  y = Conv1DTranspose(128, kernel_size=7, strides=5, padding='same')(y)
  y = LeakyReLU()(y)
  y = Conv1DTranspose(256, kernel_size=7, strides=5, padding='same')(y)
  y = LeakyReLU()(y)
  y = Conv1DTranspose(256, kernel_size=3, strides=3, padding='same')(y)
  y = LeakyReLU()(y)
  y = Conv1DTranspose(512, kernel_size=3, strides=3, padding='same')(y)
  y = LeakyReLU()(y)
  y = Conv1D(1, kernel_size=3, padding='same', activation='tanh')(y)


  model = Model(inputs = X_input, outputs = y)
  return model

In [29]:
generator = build_generator()

In [30]:
generator.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 98)]              0         
                                                                 
 dense_12 (Dense)            (None, 12544)             1241856   
                                                                 
 reshape_9 (Reshape)         (None, 98, 128)           0         
                                                                 
 conv1d_transpose_43 (Conv1D  (None, 490, 128)         114816    
 Transpose)                                                      
                                                                 
 leaky_re_lu_43 (LeakyReLU)  (None, 490, 128)          0         
                                                                 
 conv1d_transpose_44 (Conv1D  (None, 2450, 128)        114816    
 Transpose)                                                